In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
root_path = "/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2"

In [4]:
!pip install transformers
!pip install stop_words
!pip install symspellpy
!pip install language_detector 
!pip install cached_property
!pip install sentencepiece

     |████████████████████████████████| 2.3MB 13.2MB/s 
     |████████████████████████████████| 901kB 42.0MB/s 
     |████████████████████████████████| 3.3MB 68.3MB/s 
  Created wheel for stop-words: filename=stop_words-2018.7.23-cp37-none-any.whl size=32913 sha256=611b00311fe7557fc8ce7cbb3e20ee3a61d01ec014271828af9d093d71017e0a
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words
     |████████████████████████████████| 2.6MB 15.0MB/s 
  Created wheel for language-detector: filename=language_detector-5.0.2-cp37-none-any.whl size=7054 sha256=b1729e2fed911153e5e12143209221052ed1ee7b5687324c609d423a7eca5b0b
  Stored in directory: /root/.cache/pip/wheels/7d/37/fa/2098a4aa6c0d94d6ddff0d3a79669e12bc4f7baca8a760b3db
Successfully built language-detector
     |████████████████████████████████| 1.2MB 14.5MB/s 


In [5]:
import sys
sys.path.append(root_path)

In [6]:
import tensorflow as tf

from src.encoders.context_encoder_bert import ContextEncoder

from tensorflow.python import keras
import toml
import json
import pandas as pd
import numpy as np

from src.dataset.albert import AlbertDataset

## Experiment

In [7]:
sys.path.append(root_path)
from utils.experiments import get_experiments, save_results

In [8]:
# Read local `config.toml` file.
config = get_experiments('ALBERT_FINETUNE_SIMPLE')
config_df = pd.DataFrame.from_dict(config)
config_df

,bert_type,dataset_type,finetune_bert,pct_data,augment_pct,epochs
0,albert,clinical,True,1,0.1,200
1,albert,clinical,True,1,0.5,200
2,albert,clinical,True,1,1.0,200
3,albert,fiction,True,1,0.1,200
4,albert,fiction,True,1,0.5,200
5,albert,fiction,True,1,1.0,200
6,albert,wiki,True,1,0.1,200
7,albert,wiki,True,1,0.5,200
8,albert,wiki,True,1,1.0,200


In [9]:
sys.path.append(root_path)
config_df.to_csv(f'{root_path}/models/experiment.csv', header=None, index=None, sep=' ', mode='a')

In [10]:
for experiment in config:
    bert_type = experiment['bert_type']
    dataset_type = experiment['dataset_type']
    finetune_bert = experiment['finetune_bert']
    pct_data = 0.25 # experiment['pct_data']
    augment_pct = 0.25 # experiment['augment_pct']
    epochs = experiment['epochs']
    BATCH_SIZE = 64
    print("params:", bert_type, dataset_type, finetune_bert, pct_data, augment_pct, epochs)
    
    # init model
    print("initializing model...")
    model = ContextEncoder(# final_dropout=0.5,
                           dense_neurons=128,
                           bert_trainable=finetune_bert,
                           bert_type="albert-base-v2")
    
    # print("number of params: ", sum([np.prod(keras.get_value(w).shape) for w in model.trainable_weights]))
    
    # init dataset
    print("initializing dataset...")
    dataset = AlbertDataset(dataset_type=dataset_type,
                           pct_data=pct_data,
                            max_seq_length=64,
                           max_segment_length=5,
                           augment_pct=augment_pct)
    
    # process dataset
    print("processing dataset...")
    sentences, tokenized_sentences, labels = dataset.process()

    # get class weight
    neg, pos = np.bincount(labels.flatten())
    initial_bias = np.log([pos/neg])
    
    total=len(labels)
    weight_for_0 = (1 / neg)*(total)/2.0 
    weight_for_1 = (1 / pos)*(total)/2.0

    class_weight = {0: weight_for_0, 1: weight_for_1}
    print("class weight", class_weight)
    
    # create checkpoint path
    checkpoint_filepath = '{}/models/ALBERT/finetune/simple/{}-{}-{}-pct-{}-aug/checkpoint'.format(
                            root_path,
                            dataset_type,                    
                            len(sentences), 
                            pct_data,
                            augment_pct)
    print(checkpoint_filepath)
    
    # get callbacks ready.
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        monitor='val_accuracy',
        save_best_only=False,
        mode="auto",
        save_freq="epoch")

    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy', 
        verbose=1,
        patience=10,
        mode='max',
        restore_best_weights=True)

    callbacks = [
    #     early_stopping,
        model_checkpoint_callback
    ]
    
    # compiling model
    print("compiling the model...")
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=[
                      keras.metrics.BinaryAccuracy(name='accuracy')
                  ])
    
    try:
        model.load_weights(checkpoint_filepath)
        print("model loaded.")
    except:
        print("No checkpoint available.")
    
    print("starting the training process...")
    history = model.fit(dataset.format_sentences_tri_input(tokenized_sentences), 
                        tf.convert_to_tensor(labels), 
                        epochs=epochs,
                        validation_split=0.1,
                        batch_size=BATCH_SIZE,
                        verbose=1, 
                        # class_weight=class_weight,
                        callbacks=callbacks)
    
    # assigning history to experiment object for saving.
    experiment["history"] = history.history
    
    print("saving results...")
    save_results(experiment)
    break

params: albert clinical True 0.25 0.25 200
initializing model...



initializing dataset...
processing dataset...


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


class weight {0: 0.636378002528445, 1: 2.333140208574739}
/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2/models/ALBERT/finetune/simple/clinical-4027-0.25-pct-0.25-aug/checkpoint
compiling the model...
No checkpoint available.
starting the training process...
Epoch 1/200
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
57/57 [==============================] - 118s 1s/step - loss: 0.7710 - accuracy: 0.7409 - val_loss: 0.5132 - val_accuracy: 0.7916
Epoch 2/200
57/57 [==============================] - 75s 1s/step - loss: 0.5226 - accuracy: 0.7850 - val_loss: 0.5128 - val_accuracy: 0.7916
Epoch 3/200
57/57 [==============================] - 75s 1s/step - loss: 0.5252 - accuracy: 0.7850 - val_loss: 0.5122 - val_accuracy: 0.7916
Epoch 4/200
57/57 [==============================] - 75s 1s/step - loss: 0.5218 - accuracy: 0.7850 - val_loss: 0.5144 - val_accuracy: 0.7916
Epoch 5/200
57/57 [========